**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [216]:
!pip install opencv-python
!pip install scikit-video
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json, load_model
from keras.layers.core import Dense, Flatten, Dropout
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act allows the actor to choose a random action with probability $\epsilon$ and choose an action via its policy with probability $1-\epsilon$. Thus, $\epsilon$ is essential, it defines the **proportion** of the exploration of the actor and $1-\epsilon$ defines the **proportion** of the expoloitation of the actor. This strategy improves policies by guaranteeing exploration in the environment to discover new state-action pairs that can never be visited using a deterministic policy. It is then essential to find an optimal compromis between exploitation and exploration by finding an optimal value of epsilon. The value of $\epsilon$ can be modified during training such that encourage exploration over exploitation in the begining of the learning and decrease the exploration gradually. 



***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=80 # set small when debugging
epochs_test=15 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array ```position``` defines the borders of the grid, the allowed states and the position of the agent. At each step (each time tha actor take an action), -1 is assigned for the cells representing the borders (ie the states that can't be visited), 0 is assigned for all the other cells that can be visited, and 1 is assigned for the current cell occupied by the agent.

The array ```board``` defines the distribution of the rewards over each cell in the $N\times N$ grid. It stores the reward of being in each cell (x,y). each reward collected by the agent after an action is reset to 0.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        pass

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix='', printt=True, mimics = False):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
        
        while not game_over:
            # The agent performs an action
            if mimics : # we adapt the function test to the case when we use a neural network to mimics the agent ( the bonus question )
              out = agent.predict(state[None])
              action = np.argmax(out)
            else :
              action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose
        if printt :
          print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))
    return score/epochs

In [11]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 2.0/4.0. Average score (-2.0)
Win/lose count 3.0/4.0. Average score (-1.5)
Win/lose count 3.0/2.0. Average score (-0.6666666666666666)
Win/lose count 1.0/2.0. Average score (-0.75)
Win/lose count 2.5/0. Average score (-0.1)
Win/lose count 1.5/2.0. Average score (-0.16666666666666666)
Win/lose count 3.5/2.0. Average score (0.07142857142857142)
Win/lose count 2.0/2.0. Average score (0.0625)
Win/lose count 0.5/6.0. Average score (-0.5555555555555556)
Win/lose count 2.5/2.0. Average score (-0.45)
Win/lose count 0.5/5.0. Average score (-0.8181818181818182)
Win/lose count 1.5/3.0. Average score (-0.875)
Win/lose count 3.0/3.0. Average score (-0.8076923076923077)
Win/lose count 3.0/0. Average score (-0.5357142857142857)
Win/lose count 0.5/2.0. Average score (-0.6)
Final score: -0.6


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




### 1) Equation verified by $Q^{\pi}$ :
We have :
$$
\begin{align*} 
Q^\pi(s,a)&=E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
&=E_{p^{\pi}}[r(s_0,a_0) + \gamma\sum_{t=1}^{\infty}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
&=r(s,a) + \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a]\\
&=r(s,a) + E_{(s',a')\sim p(.|s,a)}\left(\gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']\right)\\
&=E_{(s',a')\sim p(.|s,a)}\left(r(s,a) + \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']\right)\\
&=E_{(s',a')\sim p(.|s,a)}\left(r(s,a) + \gamma Q^{\pi}(s',a') \right)
\end{align*}$$

### 2 ) Equation verified by $Q^*$

We recall that $Q^*(s,a)=max_\pi Q^\pi(s,a)$ and we assume that there exist an optimal policy $\pi^*$ such that $\pi^*(s)=argmax_{a}Q^*(s,a)$. We have then :

$$
\begin{align*}
Q^*(s,a) &=\max_\pi \left[E_{(s',a')\sim p(.|s,a)}\left(r(s,a) + \gamma Q^{\pi})\right(s',a') \right] \\
&= E_{s'\sim \pi^*(.|s,a),~~a'\sim \pi^*(.|s')}\left(r(s,a) + \gamma Q^{*}(s',a') \right)\\
        &= E_{s' \sim \pi^*(.|s,a)}\left(r(s,a) + \gamma \max_{a'}Q^{*}(s',a') \right)\\
\end{align*}
$$

### 3) 
We have proved that $Q^*(s,a)=E_{s' \sim \pi^*(.|s,a)}\left(r(s,a) + \gamma \max_{a'}Q^{*}(s',a') \right)$, so $Q^*$ is the solution of the equation $F(Q) = 0$ where $$F(Q) = E_{s' \sim \pi^*(.|s,a)}\left(r(s,a) + \gamma \max_{a'}Q(s',a') - Q(s,a)\right)$$

Then, minimizing $\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$ is a plausible objective to find $Q^*$.


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory)>self.max_memory:
          del self.memory[0]
        self.memory.append(m)  

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape(1,s.shape[0],s.shape[1],s.shape[2])),1)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            # Saving current state
            input_states[i] = s_
            
            # Get current Q(s,.)
            target_q[i] = self.model.predict(s_.reshape((1,s_.shape[0],s_.shape[1],s_.shape[2])))
            
            # Update Q(s,a)
            if game_over_:
                # No next state, we take only the reward
                target_q[i,a_] = r_
            else:
                # Update target_q  
                target_q[i,a_] = r_ + self.discount*np.max(self.model.predict(n_s_.reshape((1,n_s_.shape[0],n_s_.shape[1],n_s_.shape[2]))),axis=1)

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()

        model.add(Flatten(input_shape=(5,5,self.n_state)))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(4))
        ####### FILL IN
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [15]:
epochs_train = 80
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train80.mp4'))













Epoch 000/080 | Loss 0.0027 | Win/lose count 2.5/3.0 (-0.5)
Epoch 001/080 | Loss 0.0023 | Win/lose count 2.5/8.0 (-5.5)
Epoch 002/080 | Loss 0.0162 | Win/lose count 4.5/3.0 (1.5)
Epoch 003/080 | Loss 0.0113 | Win/lose count 2.5/3.0 (-0.5)
Epoch 004/080 | Loss 0.0301 | Win/lose count 4.0/1.0 (3.0)
Epoch 005/080 | Loss 0.0097 | Win/lose count 1.5/7.0 (-5.5)
Epoch 006/080 | Loss 0.0197 | Win/lose count 1.5/1.0 (0.5)
Epoch 007/080 | Loss 0.0098 | Win/lose count 4.0/4.0 (0.0)
Epoch 008/080 | Loss 0.0041 | Win/lose count 6.5/8.0 (-1.5)
Epoch 009/080 | Loss 0.0049 | Win/lose count 6.0/2.0 (4.0)
Epoch 010/080 | Loss 0.0059 | Win/lose count 3.5/4.0 (-0.5)
Epoch 011/080 | Loss 0.0543 | Win/lose count 9.0/2.0 (7.0)
Epoch 012/080 | Loss 0.0116 | Win/lose count 0.5/4.0 (-3.5)
Epoch 013/080 | Loss 0.0122 | Win/lose count 4.5/3.0 (1.5)
Epoch 014/080 | Loss 0.0024 | Win/lose count 6.5/4.0 (2.5)
Epoch 015/080 | Loss 0.0054 | Win/lose count 2.0/1.0 (1.0)
Epoch 016/080 | Loss 0.0080 | Win/los

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(16, (2,2), input_shape = (5,5,self.n_state), activation = 'relu'))
        model.add(Conv2D(32,(2,2), activation = 'relu'))
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [107]:
train_epochs = 80
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,train_epochs,prefix='cnn_train')
HTML(display_videos('cnn_train80.mp4'))

Epoch 000/080 | Loss 0.0081 | Win/lose count 3.5/7.0 (-3.5)
Epoch 001/080 | Loss 0.0123 | Win/lose count 2.0/1.0 (1.0)
Epoch 002/080 | Loss 0.0018 | Win/lose count 3.0/2.0 (1.0)
Epoch 003/080 | Loss 0.0281 | Win/lose count 2.5/1.0 (1.5)
Epoch 004/080 | Loss 0.0067 | Win/lose count 2.0/1.0 (1.0)
Epoch 005/080 | Loss 0.0060 | Win/lose count 3.5/1.0 (2.5)
Epoch 006/080 | Loss 0.0022 | Win/lose count 4.5/4.0 (0.5)
Epoch 007/080 | Loss 0.0016 | Win/lose count 4.0/0 (4.0)
Epoch 008/080 | Loss 0.0033 | Win/lose count 6.0/0 (6.0)
Epoch 009/080 | Loss 0.0019 | Win/lose count 6.0/5.0 (1.0)
Epoch 010/080 | Loss 0.0016 | Win/lose count 6.0/6.0 (0.0)
Epoch 011/080 | Loss 0.0010 | Win/lose count 9.0/2.0 (7.0)
Epoch 012/080 | Loss 0.0021 | Win/lose count 6.0/2.0 (4.0)
Epoch 013/080 | Loss 0.0033 | Win/lose count 12.5/0 (12.5)
Epoch 014/080 | Loss 0.0034 | Win/lose count 9.5/5.0 (4.5)
Epoch 015/080 | Loss 0.0017 | Win/lose count 7.5/2.0 (5.5)
Epoch 016/080 | Loss 0.0026 | Win/lose count 16.5/3.0 (13.5

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [22]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 6.0/2.0. Average score (4.0)
Win/lose count 8.5/0. Average score (6.25)
Win/lose count 12.5/5.0. Average score (6.666666666666667)
Win/lose count 8.5/3.0. Average score (6.375)
Win/lose count 15.0/4.0. Average score (7.3)
Win/lose count 13.5/5.0. Average score (7.5)
Win/lose count 19.5/4.0. Average score (8.642857142857142)
Win/lose count 12.5/1.0. Average score (9.0)
Win/lose count 18.5/3.0. Average score (9.722222222222221)
Win/lose count 20.5/2.0. Average score (10.6)
Win/lose count 6.5/4.0. Average score (9.863636363636363)
Win/lose count 10.5/4.0. Average score (9.583333333333334)
Win/lose count 9.0/4.0. Average score (9.23076923076923)
Win/lose count 11.5/2.0. Average score (9.25)
Win/lose count 11.0/4.0. Average score (9.1)
Final score: 9.1
Test of the FC
Win/lose count 9.5/3.0. Average score (6.5)
Win/lose count 15.5/1.0. Average score (10.5)
Win/lose count 5.5/3.0. Average score (7.833333333333333)
Win/lose count 9.0/2.0. Average score (7.625)
Wi

7.266666666666667

In [23]:
HTML(display_videos('cnn_test10.mp4'))

In [24]:
HTML(display_videos('fc_test10.mp4'))

In [25]:
# we try here different temperature to compare results
scores_cnn = {}
scores_fc = {}
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
for t in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
  env = Environment(grid_size=size, max_time=T,temperature=t)
  scores_cnn[t] = test(agent_cnn,env,epochs_test,prefix='cnn_test', printt = False)
  scores_fc[t] = test(agent_fc,env,epochs_test,prefix='fc_test' , printt = False)

Final score: 1.8333333333333333
Final score: 2.6666666666666665
Final score: 5.533333333333333
Final score: 4.433333333333334
Final score: 9.066666666666666
Final score: 8.766666666666667
Final score: 13.766666666666667
Final score: 10.3
Final score: 20.533333333333335
Final score: 15.666666666666666
Final score: 21.233333333333334
Final score: 21.133333333333333
Final score: 32.93333333333333
Final score: 28.066666666666666
Final score: 34.6
Final score: 33.3
Final score: 36.333333333333336
Final score: 48.86666666666667


In [73]:
import pandas as pd
scores_cnn # with respect to temperature
df = pd.DataFrame(scores_fc.keys(), columns=['temperature'])
df['scores_fc'] = scores_fc.values()
df['scores_cnn'] = scores_cnn.values()
df

,temperature,scores_fc,scores_cnn
0,0.1,2.666667,1.833333
1,0.2,4.433333,5.533333
2,0.3,8.766667,9.066667
3,0.4,10.300000,13.766667
4,0.5,15.666667,20.533333
5,0.6,21.133333,21.233333
6,0.7,28.066667,32.933333
7,0.8,33.300000,34.600000
8,0.9,48.866667,36.333333


We can see that in both case, the agent doesn't explore the map well and sometimes get struck in a region where there is no positive reward in his visibility.

We can also deduce that the cnn model works well than the fully connected network. Since our intput is an image, it's more useful to use cnn

We tested the algorithm for temperature in $\{0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9\}$ and we can see that the average score increase when the temperatur increase. For temperature greater, positive rewards cells become more numerous, which helps the agent to explore the map since more positive cells gets in his visibility.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix=''):
    #Number of won games
    score = 0
    loss = 0
    
    # Epsilon in the beginning of training
    epsilon_0 = agent.epsilon
    epsilon_final = 0.1
    #step_eps = (epsilon_0-epsilon_final)/epoch

    for e in range(epoch+1) :
      # At each epoch, we restart to a fresh game and get the initial state
      state = env.reset()
      # This assumes that the games will terminate
      game_over = False

      win = 0
      lose = 0
      # Decay for epsilon-greedy exploration
      k = 1/8
      new_eps = epsilon_0/(1+k*e)
      if new_eps > epsilon_final:
        agent.set_epsilon(new_eps)
      else :
        agent.set_epsilon(epsilon_final) 

      while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

      
      #new_eps = epsilon_0/(1+0.25*e)
      #if new_eps > epsilon_final:
        #agent.set_epsilon(new_eps)
      #else :
        #agent.set_epsilon(epsilon_final)  
        # Save as a mp4
      if e % 10 == 0:
          env.draw(prefix+str(e))

        # Update stats
      score += win-lose

      print("Epsilon {:.4f} | Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(agent.epsilon, e, epoch, loss, win, lose, win-lose))
      agent.save(name_weights=prefix+'new_model.h5',name_model=prefix+'new_model.json')

class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):

        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        ## In Environment exploring:
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        self.malus_position[self.x, self.y] = 0.1
        if train:
          reward = -self.malus_position[self.x, self.y]

        reward +=  self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.malus_position[self.x, self.y] = 0.1
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    

In [111]:
# Training DQN_FC
epochs_train = 90
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent_fc = DQN_FC(size, lr=.1, epsilon = 0.7, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent_fc, env, epochs_train, prefix='fc_train_explore')
HTML(display_videos('fc_train_explore80.mp4'))

Epsilon 0.7000 | Epoch 000/090 | Loss 0.0179 | Win/lose count 6.000000000000001/31.60000000000008 (-25.60000000000008)
Epsilon 0.6222 | Epoch 001/090 | Loss 0.0175 | Win/lose count 3.9999999999999996/25.10000000000008 (-21.10000000000008)
Epsilon 0.5600 | Epoch 002/090 | Loss 0.0091 | Win/lose count 9.200000000000003/25.800000000000047 (-16.600000000000044)
Epsilon 0.5091 | Epoch 003/090 | Loss 0.0078 | Win/lose count 9.200000000000003/25.800000000000047 (-16.600000000000044)
Epsilon 0.4667 | Epoch 004/090 | Loss 0.0237 | Win/lose count 6.400000000000001/26.500000000000124 (-20.100000000000122)
Epsilon 0.4308 | Epoch 005/090 | Loss 0.0146 | Win/lose count 7.200000000000002/24.30000000000007 (-17.10000000000007)
Epsilon 0.4000 | Epoch 006/090 | Loss 0.0138 | Win/lose count 10.400000000000004/25.500000000000032 (-15.100000000000028)
Epsilon 0.3733 | Epoch 007/090 | Loss 0.0133 | Win/lose count 10.000000000000004/26.600000000000065 (-16.600000000000062)
Epsilon 0.3500 | Epoch 008/090 | Lo

In [112]:
# Training DQN_CNN
epochs_train = 90
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.7, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent_cnn, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore80.mp4'))

Epsilon 0.7000 | Epoch 000/090 | Loss 0.0174 | Win/lose count 8.800000000000002/37.900000000000155 (-29.10000000000015)
Epsilon 0.6222 | Epoch 001/090 | Loss 0.0084 | Win/lose count 7.200000000000002/23.30000000000006 (-16.10000000000006)
Epsilon 0.5600 | Epoch 002/090 | Loss 0.0097 | Win/lose count 10.000000000000004/23.600000000000055 (-13.600000000000051)
Epsilon 0.5091 | Epoch 003/090 | Loss 0.0100 | Win/lose count 14.000000000000007/33.600000000000094 (-19.600000000000087)
Epsilon 0.4667 | Epoch 004/090 | Loss 0.0052 | Win/lose count 5.6000000000000005/23.70000000000005 (-18.100000000000048)
Epsilon 0.4308 | Epoch 005/090 | Loss 0.0041 | Win/lose count 10.800000000000004/26.400000000000084 (-15.60000000000008)
Epsilon 0.4000 | Epoch 006/090 | Loss 0.0390 | Win/lose count 11.600000000000005/22.20000000000002 (-10.600000000000016)
Epsilon 0.3733 | Epoch 007/090 | Loss 0.0059 | Win/lose count 12.400000000000006/24.00000000000004 (-11.600000000000033)
Epsilon 0.3500 | Epoch 008/090 | 

In [116]:
# Evaluation DQN_FC
test(agent_fc,env,epochs_test,prefix='fc_test_explore')
HTML(display_videos('fc_test_explore10.mp4'))

Win/lose count 13.0/3.0. Average score (10.0)
Win/lose count 20.0/4.0. Average score (13.0)
Win/lose count 8.5/1.0. Average score (11.166666666666666)
Win/lose count 15.5/1.0. Average score (12.0)
Win/lose count 9.5/1.0. Average score (11.3)
Win/lose count 16.5/4.0. Average score (11.5)
Win/lose count 13.5/2.0. Average score (11.5)
Win/lose count 13.5/5.0. Average score (11.125)
Win/lose count 6.0/0. Average score (10.555555555555555)
Win/lose count 17.0/2.0. Average score (11.0)
Win/lose count 21.0/2.0. Average score (11.727272727272727)
Win/lose count 14.0/1.0. Average score (11.833333333333334)
Win/lose count 18.5/1.0. Average score (12.26923076923077)
Win/lose count 17.0/1.0. Average score (12.535714285714286)
Win/lose count 9.5/2.0. Average score (12.2)
Final score: 12.2


In [134]:
# Evaluation DQN_CNN
test(agent_cnn,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 17.0/2.0. Average score (15.0)
Win/lose count 17.5/1.0. Average score (15.75)
Win/lose count 21.5/4.0. Average score (16.333333333333332)
Win/lose count 23.0/5.0. Average score (16.75)
Win/lose count 24.5/10.0. Average score (16.3)
Win/lose count 19.5/2.0. Average score (16.5)
Win/lose count 18.0/5.0. Average score (16.0)
Win/lose count 21.5/5.0. Average score (16.0625)
Win/lose count 18.5/0. Average score (16.333333333333332)
Win/lose count 10.5/1.0. Average score (15.65)
Win/lose count 26.0/1.0. Average score (16.5)
Win/lose count 24.0/1.0. Average score (17.041666666666668)
Win/lose count 20.5/1.0. Average score (17.23076923076923)
Win/lose count 16.0/4.0. Average score (16.857142857142858)
Win/lose count 24.0/6.0. Average score (16.933333333333334)
Final score: 16.933333333333334


We can see with these two ideas, the models work better. The average score in the test was improved from 9.1 to **16.93** for the DQN using a cnn model and from 7.26 to **12.2** for the Fc model

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

To train a model that mimicks our best DQN model, we will first generate 10000 wining games. After that we will extract each state with shape $(5,5,3)$ to add it to X the input of our model, and each action taken by the agent that corresponds to this state and add it to y. Finally, we get a dataset where X contains state samples and y contains corresponds actions taken by the agent. After that, we will train a cnn model to mimicks these actions and test it as we tested the other agent.

In [0]:
import copy
from keras.utils import to_categorical

def generate_games(n, env, agent):
  games = {}
  X = []
  y = []
  i= 0
  while len(games)< n :
    game = []
    actions = []
    state = env.reset()
    game_over = False
    win = 0
    while not game_over:
      ex_state = state.copy()
      action = agent.act(state)
      state, reward, game_over = env.act(action)
            # Update the counters
      if reward > 0:
        win = win + reward
        game.append(ex_state)
        X.append(ex_state)
        y.append(to_categorical(action,num_classes=4).reshape(4,))
        actions.append(to_categorical(action,num_classes=4).reshape(4,))
    games[i] = {'game': game, 'action': actions}   
    i+=1
    if len(games)%200 == 0 : print('{} games simulated'.format(len(games)))
  return games, np.array(X),np.array(y)

In [222]:
n_games = 2000
games, X, y = generate_games(n_games, env, agent_cnn)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split( X, y, test_size=0.3, random_state=42)
print('Shape of X train', X_train.shape)
print('Shape of y train', y_train.shape)
print('Shape of X test', X_val.shape)
print('Shape of y test', y_val.shape)

200 games simulated
400 games simulated
600 games simulated
800 games simulated
1000 games simulated
1200 games simulated
1400 games simulated
1600 games simulated
1800 games simulated
2000 games simulated
Shape of X train (50276, 5, 5, 3)
Shape of y train (50276, 4)
Shape of X test (21548, 5, 5, 3)
Shape of y test (21548, 4)


In [282]:

model = Sequential()
model.add(Conv2D(16, (2,2), input_shape = (5,5,3), activation = 'relu'))
model.add(Dropout(0.2))
model.add(Conv2D(32, (2,2),activation = 'relu'))
#model.add(MaxPooling2D((1,1)))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(y_train.shape[1], activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',  metrics = ['accuracy'] )

print(model.summary())

Model: "sequential_68"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_118 (Conv2D)          (None, 4, 4, 16)          208       
_________________________________________________________________
dropout_40 (Dropout)         (None, 4, 4, 16)          0         
_________________________________________________________________
conv2d_119 (Conv2D)          (None, 3, 3, 32)          2080      
_________________________________________________________________
flatten_49 (Flatten)         (None, 288)               0         
_________________________________________________________________
dense_67 (Dense)             (None, 128)               36992     
_________________________________________________________________
dense_68 (Dense)             (None, 4)                 516       
Total params: 39,796
Trainable params: 39,796
Non-trainable params: 0
_________________________________________________

In [283]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='mimic_model.hdf5', verbose=1,
                                               save_best_only=True, monitor='val_acc')
bs = 32
n_epochs = 5
history = model.fit(X_train, y_train, batch_size=bs, nb_epoch=n_epochs, validation_data=(X_val, y_val),callbacks=[checkpointer])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


Train on 50276 samples, validate on 21548 samples
Epoch 1/5
50276/50276 [==============================] - 14s 281us/step - loss: 0.2211 - acc: 0.9114 - val_loss: 0.1316 - val_acc: 0.9483

Epoch 00001: val_acc improved from -inf to 0.94830, saving model to mimic_model.hdf5
Epoch 2/5
50276/50276 [==============================] - 11s 217us/step - loss: 0.1449 - acc: 0.9413 - val_loss: 0.1246 - val_acc: 0.9519

Epoch 00002: val_acc improved from 0.94830 to 0.95192, saving model to mimic_model.hdf5
Epoch 3/5
50276/50276 [==============================] - 11s 211us/step - loss: 0.1336 - acc: 0.9473 - val_loss: 0.1142 - val_acc: 0.9590

Epoch 00003: val_acc improved from 0.95192 to 0.95898, saving model to mimic_model.hdf5
Epoch 4/5
50276/50276 [==============================] - 11s 212us/step - loss: 0.1252 - acc: 0.9513 - val_loss: 0.1223 - val_acc: 0.9514

Epoch 00004: val_acc did not improve from 0.95898
Epoch 5/5
50276/50276 [==============================] - 11s 225us/step - loss: 0.1

In [284]:
# Evaluation mimics model
model = load_model('mimic_model.hdf5')
test(model,env,epochs_test,prefix='mimic_test', mimics=True)
HTML(display_videos('mimic_test10.mp4'))

Win/lose count 19.5/2.0. Average score (17.5)
Win/lose count 20.5/2.0. Average score (18.0)
Win/lose count 19.0/3.0. Average score (17.333333333333332)
Win/lose count 3.0/1.0. Average score (13.5)
Win/lose count 11.5/0. Average score (13.1)
Win/lose count 21.5/6.0. Average score (13.5)
Win/lose count 14.0/0. Average score (13.571428571428571)
Win/lose count 15.0/1.0. Average score (13.625)
Win/lose count 3.5/0. Average score (12.5)
Win/lose count 6.5/1.0. Average score (11.8)
Win/lose count 25.5/3.0. Average score (12.772727272727273)
Win/lose count 27.5/1.0. Average score (13.916666666666666)
Win/lose count 14.0/0. Average score (13.923076923076923)
Win/lose count 15.5/0. Average score (14.035714285714286)
Win/lose count 24.0/6.0. Average score (14.3)
Final score: 14.3


It didn't improve really the model. The QDN CNN works better

***